In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
pd. set_option("display.max_columns", None) 

In [3]:
df = pd.read_csv('./android-fitness-and-medical-cleaned-data(no-duplicate).csv')

#### FEATURE ENGINNERING

**Rating Count Categorization**

1+, 5+, 10+, 50+, 100+, 500+, 1,000+, 5,000+, 10,000+, 50,000+, 100,000+, 500,000+, 1,000,000+, 5,000,000, 10,000,000+, 50,000,000+

In [4]:
def determine_if_app_is_free_or_not(price):
    if (price > 0):
        return 'PAID'
    return 'FREE'

In [5]:
def grade_app_with_rating(app_rating):
    if ((app_rating >=0) and (app_rating <2.5)):
        return 'BAD'
    if ((app_rating >=2.5) and (app_rating <3.0)):
        return 'FAIR'
    if ((app_rating >=3.0) and (app_rating <4.0)):
        return 'GOOD'
    if ((app_rating >=4.0) and (app_rating <4.5)):
        return 'VERY GOOD'
    if (app_rating >=4.5):
        return 'EXCELLENT'

In [6]:
def create_rating_count_range(rating_count):
    if (rating_count >=1 and rating_count<5):
        return '1+'
    if (rating_count >=5 and rating_count<10):
        return '5+'
    if (rating_count >=10 and rating_count<50):
        return '10+'
    if (rating_count >=50 and rating_count<100):
        return '50+'
    if (rating_count >=100 and rating_count<500):
        return '100+'
    if (rating_count >=500 and rating_count<1000):
        return '500+'
    if (rating_count >=1000 and rating_count<5000):
        return '1,000+'
    if (rating_count >=5000 and rating_count<10000):
        return '5,000+'
    if (rating_count >=10000 and rating_count<50000):
        return '10,000+'
    if (rating_count >=50000 and rating_count<100000):
        return '50,000+'
    if (rating_count >=100000 and rating_count<500000):
        return '100,000+'
    if (rating_count >=500000 and rating_count<1000000):
        return '500,000+'
    if (rating_count >=1000000 and rating_count<5000000):
        return '1,000,000+'
    if (rating_count >=5000000 and rating_count<10000000):
        return '5,000,000+'
    if (rating_count >=10000000 and rating_count<50000000):
        return '10,000,000+'
    if (rating_count >=50000000 and rating_count<100000000):
        return '50,000,000+'
    return 0

In [7]:
def is_app_updated_in_2022(last_updated_date, released_date, version):
    if (version == '1.0.0'):
        return False
    if (last_updated_date == released_date):
        return False
    if '2022' in last_updated_date:
        return True
    return False

In [8]:
def is_app_ever_updated(last_updated_date, released_date, version):
    if (version == '1.0.0'):
        return False
    if (last_updated_date == released_date):
        return False
    return True

In [9]:
def is_app_version_device_dependent(version):
    return version == 'Varies with device'

In [10]:
def is_app_available_in_multiple_countries(count):
    return count > 1

In [11]:
def perform_feature_engineering(df):
    df['Service Type'] = df['Price'].apply(determine_if_app_is_free_or_not)
    df['is_available_in_multiple_countries'] = df['Country Count'].apply(is_app_available_in_multiple_countries)
    df['is_version_app_dependent'] = df['Version'].apply(is_app_version_device_dependent)
    df['Rating Count Range'] = df['Rating Count'].apply(create_rating_count_range)
    df['App Grade'] = df['Average Rating'].apply(grade_app_with_rating)
    for index in df.index:
        link = df.at[index, 'App Link']
        last_updated_date = df.at[index, 'Last Updated']
        released_date = df.at[index, 'Released Date']
        version = df.at[index, 'Version']
        df.at[index, 'is_updated_this_year'] = is_app_updated_in_2022(last_updated_date, released_date, version)
        df.at[index, 'is_app_ever_updated'] = is_app_ever_updated(last_updated_date, released_date, version)

In [12]:
perform_feature_engineering(df)

In [13]:
df[:5]

,Unnamed: 0,App Name,Age Rating,Price,InApp Purchase,Average Rating,Rating Count,App Link,Last Updated,Released Date,Downloads,Version,Application Type,Country Count,Countries,Service Type,is_available_in_multiple_countries,is_version_app_dependent,Rating Count Range,App Grade,is_updated_this_year,is_app_ever_updated
0,0,TraceTogether,PEGI 3,0.00,False,2.9,57100,https://play.google.com/store/apps/details?id=...,"May 31, 2022","Mar 9, 2020","1,000,000+",2.14.2,MEDICAL,31,"FO,HR,SM,GI,EE,SI,GL,SK,PF,SJ,IS,VA,BG,WF,LI,A...",FREE,True,False,"50,000+",FAIR,True,True
1,1,Blood Pressure Analyze,PEGI 3,0.00,True,3.9,12300,https://play.google.com/store/apps/details?id=...,"Sep 9, 2021","Apr 1, 2019","1,000,000+",1.3.2,MEDICAL,32,"FO,HR,SM,GI,EE,SI,GL,SK,PF,SJ,IS,VA,BG,WF,LI,A...",FREE,True,False,"10,000+",GOOD,False,True
2,2,Monash University FODMAP diet,PEGI 3,5.60,False,3.5,2470,https://play.google.com/store/apps/details?id=...,"Jan 18, 2022","Nov 6, 2013","100,000+",3.0.9,MEDICAL,46,"FO,PT,HR,IT,DE,FI,EE,HU,SI,ES,GL,IE,RO,SK,PF,S...",PAID,True,False,"1,000+",GOOD,True,True
3,3,Squeezy: NHS Pelvic Floor App,PEGI 3,2.99,False,4.6,3250,https://play.google.com/store/apps/details?id=...,"Mar 4, 2019","Sep 17, 2014","50,000+",Varies with device,MEDICAL,45,"FO,PT,HR,SM,GI,FI,EE,HU,SI,GL,IE,RO,SK,PF,SJ,I...",PAID,True,True,"1,000+",EXCELLENT,False,True
4,4,Muscle Trigger Point Anatomy,PEGI 3,2.69,False,4.5,1590,https://play.google.com/store/apps/details?id=...,"Jul 30, 2019","Jan 27, 2011","50,000+",2.4.8,MEDICAL,42,"FO,PT,HR,IT,DE,FI,EE,HU,SI,GL,IE,RO,SK,PF,SJ,I...",PAID,True,False,"1,000+",EXCELLENT,False,True


#### VISUALIZATION